# RAG foodstuff

## Imports

In [1]:
import torch
import pandas as pd

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableMap

## Data

In [2]:
df = pd.read_csv('data/data_clear.csv', index_col=0)
df

,title,price,category,subcategory,weight,energy_value,proteins,fats,carbs,composition
0,соус салатный kuhne honey mustard горчично-мед...,374 ₽/шт,бакалея,супермаркет,250 мл,161.00,0.80,11.0,14.0,"вода, глюкозно-фруктозный сироп, масло рапсово..."
1,рулетики из баклажанов с сырной начинкой,228 ₽/шт,кухни мира,готовая еда,80 г,317.90,9.70,29.5,3.4,"баклажан запеченный (баклажан свежий, масло по..."
2,соус-приправа кинто наршараб гранатовый с/б,310 ₽/шт,бакалея,супермаркет,245 г,250.00,0.00,0.0,62.0,"концентрат гранатового сока, сахар, вода, регу..."
3,суп «мисо-рамен» с морепродуктами,498 ₽/шт,супы,готовая еда,580 г,45.40,6.90,1.0,2.2,"бульон рамен (вода питьевая, мисо паста (соевы..."
4,"тунец (полосатый) филе натуральный,",275 ₽/шт,рыбные консервы,"рыба, икра и морепродукты",170 г,91.76,21.14,0.8,NaN,"тунец полосатый, вода питьевая, соль продукция..."
...,...,...,...,...,...,...,...,...,...,...
5447,батончик светлый с клубникой (веган),158 ₽/шт,шоколад и батончики,сладости и десерты,50 г,515.00,13.00,35.0,37.0,"ядра арахиса жареные, сахар тростниковый нераф..."
5448,рис националь краснодарский,350 ₽/шт,бакалея,супермаркет,1.5 кг,360.00,6.50,0.5,83.0,"крупа рисовая шлифованная ""рис краснодарский"""
5449,"суп «гаспачо» (испания),",210 ₽/шт,супы и заготовки для супа,консервация,330 мл,41.40,0.80,2.4,3.6,"свежие овощи (93%) (помидоры, перец сладкий, о..."
5450,плоды каперсов federici,335 ₽/шт,консервация и соленья,супермаркет,230 г,22.70,1.30,0.3,1.4,"плоды каперсов, вода питьевая, уксус, соль."


## FAISS

In [3]:
embedding_model = HuggingFaceEmbeddings(
    model_name='cointegrated/LaBSE-en-ru', 
    model_kwargs={'device': 'cuda'},
)

In [4]:
def format_text(row):
    """объединение текста"""
    return (
        f"Название: {row['title']}\n"
        f"Цена: {row['price']}\n"
        f"Вес/Объем: {row['weight']}\n"
        f"Энергетическая ценность: {row['energy_value']} ккал\n"
        f"Белки: {row['proteins']} г, Жиры: {row['fats']} г, Углеводы: {row['carbs']} г\n"
        f"Категория: {row['category']}\n"
        f"Подкатегория: {row['subcategory']}\n"
        f"Состав: {row['composition']}"
    )

print(format_text(df.loc[0]))

Название: соус салатный kuhne honey mustard горчично-медовый 
Цена: 374 ₽/шт
Вес/Объем: 250 мл
Энергетическая ценность: 161.0 ккал
Белки: 0.8 г, Жиры: 11.0 г, Углеводы: 14.0 г
Категория: бакалея
Подкатегория: супермаркет
Состав: вода, глюкозно-фруктозный сироп, масло рапсовое рафинированное, горчица 7 % (вода, семена горчицы, спиртовой уксус, соль, куркума, гвоздика), белый винный уксус, мед 3 %, стабилизатор: дикрахмалфосфат оксипропилированный; соль, семена горчицы грубого помола, яичный желток, загуститель: ксантановая камедь; специи, красители: бета-каротин, экстракт паприки


### Get vector base

In [5]:
# documents = [Document(page_content=format_text(row), metadata={"index": i}) for i, row in df.iterrows()]
# vector_store = FAISS.from_documents(documents, embedding_model)
# vector_store.save_local('data/faiss_products_db')

### Load vector base

In [6]:
vector_store = FAISS.load_local('data/faiss_products_db', embedding_model, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={"k": 15})

In [7]:
query = """Список доступных продуктов:
- Куриная грудка
- Яйца
- Овсянка"""
result = retriever.invoke(query)

for doc in result:
    text = doc.page_content[:200]
    print(f"{text}...")
    print('='*30)

Название: бескостное мясо бедра цыпленка-бройлера для гриля «пикантное», зам.
Цена: 645 ₽/шт
Вес/Объем: 800 г
Энергетическая ценность: 169.6 ккал
Белки: 16.0 г, Жиры: 10.8 г, Углеводы: 2.1 г
Категория...
Название: тефтели из индейки и говядины
Цена: 404 ₽/шт
Вес/Объем: 450 г
Энергетическая ценность: 182.8 ккал
Белки: 16.4 г, Жиры: 9.2 г, Углеводы: 8.6 г
Категория: котлеты и полуфабрикаты
Подкатегория:...
Название: котлеты говядина-свинина с капустой в панировке, зам.
Цена: 465 ₽/шт
Вес/Объем: 600 г
Энергетическая ценность: 124.7 ккал
Белки: 13.3 г, Жиры: 3.5 г, Углеводы: 10.0 г
Категория: котлеты, наг...
Название: колбаса вареная с окороком котто, 
Цена: 395 ₽/шт
Вес/Объем: 300 г
Энергетическая ценность: 266.4 ккал
Белки: 12.2 г, Жиры: 23.2 г, Углеводы: 2.2 г
Категория: колбаса варёная и ветчина
Подка...
Название: пельмени «фермерские» из индейки, зам.
Цена: 395 ₽/шт
Вес/Объем: 400 г
Энергетическая ценность: 179.6 ккал
Белки: 15.5 г, Жиры: 3.2 г, Углеводы: 22.2 г
Категория: пельмени, в

/home/lake/develop/rag_foodstuff/.venv/lib/python3.11/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  return F.linear(input, self.weight, self.bias)


## LLM

In [19]:
llm = ChatOllama(model='infidelis/GigaChat-20B-A3B-instruct-v1.5:q4_K_M', temperature=.6)

In [9]:
def format_docs(docs):
    return "\n\n".join([f"- {d.page_content}" for d in docs])

In [10]:
system_template = """
Ты - эксперт по питанию и диетологии. Твоя задача — помогать пользователю в составления плана питания на основе переданного контекста.

Правила составления меню:
1. Используй только те продукты, которые переданы в контексте.
2. Учитывай разнообразие: сочетай белки, жиры и углеводы в каждом приеме пищи.
3. Оптимизируй рацион по калорийности и нутриентам, чтобы он был сбалансированным.
4. Учитывай предпочтения пользователя, если они указаны в запросе.
5. Если в контексте недостаточно ингредиентов для полноценного меню, предложи альтернативы или упомяни, что можно добавить.
6. Меню должно быть простым в приготовлении и состоять из доступных ингредиентов.

Формат меню:
- Завтрак: *примерное меню*
- Обед: *примерное меню*
- Ужин: *примерное меню*
"""


human_template = """
Контекст:
{context}

Запрос пользователя:
{question}

Ответ:
"""

template = ([
    ("system", system_template),
    ("human", human_template)
])

chain = (
    RunnableMap({"context": retriever | format_docs, "question": RunnablePassthrough()})
    | ChatPromptTemplate.from_messages(template)
    | llm
)

In [28]:
def query_rag(text: str):
    for chunk in chain.stream(text):
        print(chunk.content, end='', flush=True) 

### Sample queries

In [ ]:
query = "Составь диетический рацион на день на период сушки. Коллораж не должен превышать 2400ккал. Приведи краткое обоснование твоего выбора."
query_rag(query)

### Завтрак (7:00)
- **Овсянка с протеиновым порошком**:
  - Овсяные хлопья – 50 г
  - Вода или молоко обезжиренное
  - Протеиновый порошок (по желанию) – 30 г
  - Фрукты, например, яблоко, банан или ягоды – 50 г

### Перекус (10:00)
- **Творог с орехами и ягодами**:
  - Творог обезжиренный – 100 г
  - Орехи грецкие/миндаль – 20 г
  - Ягоды (клубника, черника) – 50 г

### Обед (13:00)
- **Куриная грудка с чечевицей**:
  - Куриная грудка без кожи и костей – 150 г
  - Чечевица красная – 80 г сухого веса
  - Овощной салат из помидоров, огурцов, зелени – 100 г

### Перекус (16:00)
- **Протеиновый коктейль**:
  - Протеиновый порошок (например, сывороточный) – 30 г
  - Вода или молоко обезжиренное – 250 мл
  - Банан – 50 г

### Ужин (19:00)
- **Говяжий стейк с брокколи**:
  - Говяжий стейк (нежирный) – 180 г
  - Брокколи – 200 г
  - Кабачки или цветная капуста – 150 г

### Перекус перед сном (22:00)
- **Творог с миндалем**:
  - Творог обезжиренный – 100 г
  - Миндаль – 20 г

### Обоснование 

In [ ]:
query = "Я активно занимаюсь спортом и хочу набрать массу."
query_rag(query)

### Завтрак:
1. Овсяная каша с добавлением протеинового порошка, банана и орехов.
2. Яичница из 4-5 яиц с добавлением овощей (помидоры, шпинат) и кусочка цельнозернового хлеба.
3. Смузи на основе молока или йогурта с добавлением овсянки, фруктов и орехов для дополнительного источника белка.

### Перекус:
1. Горсть миндаля или других орехов.
2. Йогурт с фруктами и медом.

### Обед:
1. Куриная грудка, запеченная с овощами (брокколи, морковь) и коричневым рисом.
2. Стейк из говядины с картофельным пюре и салатом из свежих овощей.
3. Лосось на гриле с овощами-гриль и цельнозерновым хлебом.

### Полдник:
1. Творог с медом и ягодами.
2. Греческий йогурт с ореховой пастой и фруктами.

### Ужин:
1. Говяжий стейк с большим количеством овощей, приготовленных на пару или гриле.
2. Рыба (лосось, тунец) с рисом басмати и салатом из свежих овощей.
3. Паста из твердых сортов пшеницы с овощным соусом и куриной грудкой.

### Перед сном:
1. Протеиновый коктейль или творог для поддержания положительного 

In [ ]:
query = "Я вегетарианец. Составь меню на день без мяса и рыбы."
query_rag(query)

**Завтрак:** Овсяная каша с фруктами и орехами
- Овсянка – 50 г (сухой вес)
- Вода или молоко (растительное, например миндальное) – 150 мл
- Яблоко – ½ шт.
- Банан – ½ шт.
- Орехи (грецкие, миндаль) – горсть

**Обед:** Овощной салат с киноа и авокадо
- Киноа – 70 г (сухой вес)
- Авокадо – ¼ шт.
- Огурец – ½ шт.
- Помидор – 1 шт.
- Зелень (петрушка, укроп) – небольшой пучок
- Оливковое масло – 1 ст. ложка

**Ужин:** Тофу с овощами на гриле
- Тофу – 150 г
- Болгарский перец – ½ шт.
- Кабачок – ¼ шт.
- Брокколи – 2 соцветия
- Оливковое масло – 1 ст. ложка

Приятного аппетита!


In [ ]:
query = """Список доступных продуктов:
- Куриная грудка
- Яйца
- Овсянка
- Яблоки
- Брокколи
- Рис
- Молоко
- Греческий йогурт

Составь меню на день с акцентом на белки."""

query_rag(query)

Меню на день, ориентированное на потребление достаточного количества белка:

**Завтрак:**
Овсянка (150 г сухой овсянки) с нарезанными яблоками и брокколи (по 1 шт.), заправленная греческим йогуртом (200 мл). Можно добавить немного меда или орехов по вкусу.

**Перекус:**
Куриная грудка (150-200 г) без кожи, запеченная в духовке с травами и специями. К ней можно подать брокколи на пару (около 150 г).

**Обед:**
Грудка куриная (150 г), приготовленная на гриле или отварная, подается с рисом (100 г) и брокколи (200 г). Можно добавить немного оливкового масла для аромата.

**Перекус второй половины дня (по желанию):**
Яйцо вареное (1 шт.), салат из брокколи (50 г), заправленный греческим йогуртом или оливковым маслом, без добавления соли. 

**Ужин:**
Грудка куриная (200 г) тушеная с овощами, такими как лук и болгарский перец – добавляйте овощи по своему вкусу и предпочтению.

Если вам нужно больше рекомендаций или разнообразия в вашем меню на следующий день, пожалуйста, уточните ваши предпоч

In [ ]:
query = "Составь недорогое меню на день. Общая цена продуктов не должна превышать 500р"
query_rag(query)

Завтрак: 
- Макаронные изделия "Игрушки" - 127 ₽ за 500 г (3 порций по 150 г на сумму 127 ₽)
   * 150 г макарон = 42 ₽
   * 1 яйцо = около 8-10 ₽ в зависимости от региона
   * Соль, перец по вкусу
   * Итого: 42 ₽ + 10 ₽ (яйцо) ≈ 52 ₽

Рецепт:
1. Отварить макароны согласно инструкции на упаковке.
2. Взбить яйцо с солью и перцем.
3. Обжарить яйцо на сковороде до готовности.
4. Смешать готовую пасту с яйцом.

Обед: 
- Скумбрия без головы "Русское море" - 399 ₽/шт (1 шт = 300 г)
   * 1 скумбрия = 266 ₽
   * Лук белый - 160 ₽ за 500 г (1 луковица ≈ 40-50 ₽, используем примерно четверть)
   * Укроп свежий - около 20-30 ₽/кг (достаточно небольшой пучок для аромата)
   * Лимон - 1 шт = 20-30 ₽
   * Итого: 266 ₽ + 40 ₽ (лук) ≈ 300 ₽

Рецепт:
1. Разделать скумбрию, удалить кости и кожу.
2. Нарезать лук и укроп, сбрызнуть лимонным соком.
3. Запечь рыбу в духовке при 180°C около 25-30 минут до готовности.
4. Подавать с нарезанным лимоном и зеленью.

Ужин: 
- Нут продовольственный - 143 ₽ за 500 г

In [ ]:
query = "Составь недорогое меню на день. Общая цена продуктов не должна превышать 500р"
query_rag(query)

**Завтрак:**  
- Овсяная каша (100 г овсянки, 200 мл молока, банан) – 60 ₽  
- Тосты с арахисовой пастой и медом – 30 ₽  
**Итого: 90 ₽**

**Обед:**  
- Скумбрия на гриле (1 скумбрия, лимонный сок, соль, перец) – 150 ₽  
- Салат из огурцов и помидоров с оливковым маслом – 20 ₽  
- Хлебцы – 10 ₽  
**Итого: 180 ₽**

**Ужин:**  
- Макароны «бантики» (300 г, 100 г куриного фарша, сыр пармезан) – 200 ₽  
**Итого: 200 ₽**

**Общий итог:**   470 ₽

Если добавить немного зелени или овощей к каждому приему пищи, можно уложиться в бюджет. Это примерное меню на день, которое обеспечит вас всеми необходимыми питательными веществами и не превысит лимит в 500₽.
